# Getting the models to merge in Checkpoint format


In [ ]:
!wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix/AbyssOrangeMix_base.ckpt

--2023-08-03 21:56:25--  https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix/AbyssOrangeMix_base.ckpt
Resolving huggingface.co (huggingface.co)... 99.84.108.87, 99.84.108.55, 99.84.108.129, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.87|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a3/2d/a32dc75b6b955f4b9a0498061b68d02fc24b9e596c8126c9817795c3f0356c18/bbd2621f3ec4fad707f75fc032a2c2602c296180a53ed3d9897d8ca7a01dd6ed?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27AbyssOrangeMix_base.ckpt%3B+filename%3D%22AbyssOrangeMix_base.ckpt%22%3B&Expires=1691358985&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MTM1ODk4NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9hMy8yZC9hMzJkYzc1YjZiOTU1ZjRiOWEwNDk4MDYxYjY4ZDAyZmMyNGI5ZTU5NmM4MTI2Yzk4MTc3OTVjM2YwMzU2YzE4L2JiZDI2MjFmM2VjNGZhZDcwN2Y3NWZjMDMyYTJjMj

In [ ]:
!wget https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt

--2023-08-03 21:58:17--  https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt
Resolving huggingface.co (huggingface.co)... 99.84.108.55, 99.84.108.87, 99.84.108.129, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/83/8e/838e890bec8fddfdbc93ceb156ced8b290b4de1f1116c58b34d3af4c9f4fcea7/9dade826203c7ee369881b5dc20d34298fa644c1f137568e09fbc4b9a0d3e817?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27wd-v1-3-float32.ckpt%3B+filename%3D%22wd-v1-3-float32.ckpt%22%3B&Expires=1691359097&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MTM1OTA5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84My84ZS84MzhlODkwYmVjOGZkZGZkYmM5M2NlYjE1NmNlZDhiMjkwYjRkZTFmMTExNmM1OGIzNGQzYWY0YzlmNGZjZWE3LzlkYWRlODI2MjAzYzdlZTM2OTg4MWI1ZGMyMGQzNDI5OGZhNjQ0YzFmMTM3NTY4ZTA5ZmJjN

# Merging the models

In [ ]:
import os

import torch
from tqdm import tqdm

without_vae = True

output = "merged_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model_0 = torch.load("/content/AbyssOrangeMix_base.ckpt",map_location=torch.device('cuda'))
theta_0 = model_0["state_dict"]
model_1 = torch.load("/content/wd-v1-3-float32.ckpt",map_location=torch.device('cuda'))
theta_1 = model_1
alpha = 0.7

output_file = f'{output}-{str(alpha)[2:] + "0"}.ckpt'

# check if output file already exists, ask to overwrite
if os.path.isfile(output_file):
    print("Output file already exists. Overwrite? (y/n)")
    while True:
        overwrite = input()
        if overwrite == "y":
            break
        elif overwrite == "n":
            print("Exiting...")
            exit()
        else:
            print("Please enter y or n")


for key in tqdm(theta_0.keys(), desc="Stage 1/2"):
    # skip VAE model parameters to get better results(tested for anime models)
    # for anime model，with merging VAE model, the result will be worse (dark and blurry)
    if without_vae and "first_stage_model" in key:
        continue

    if "model" in key and key in theta_1:
        theta_0[key] = (1 - alpha) * theta_0[key] + alpha * theta_1[key]

for key in tqdm(theta_1.keys(), desc="Stage 2/2"):
    if "model" in key and key not in theta_0:
        theta_0[key] = theta_1[key]

print("Saving...")

torch.save({"state_dict": theta_0}, output_file)

print("Done!")

Stage 2/2: 100%|██████████| 7/7 [00:00<00:00, 67185.65it/s]


Saving...
Done!
